# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
os.chdir("../")   # change directory to location of api_key

from pprint import pprint
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [2]:
# read cities from Watcsv and place in data frome

cityDataDF = pd.read_csv("Weatherpy/output_data/cities.csv")
cityDataDF.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,illoqqortoormiut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Hilo,19.7297,-155.0900,80.28,87.0,75.0,4.61,US,1.660460e+09
2,2,Punta Arenas,-53.1500,-70.9167,32.11,93.0,75.0,8.05,CL,1.660460e+09
3,3,Mataura,-46.1927,168.8643,50.49,88.0,3.0,4.16,NZ,1.660460e+09
4,4,Strezhevoy,60.7333,77.5889,57.29,71.0,100.0,18.05,RU,1.660460e+09


In [ ]:
len(cityDataDF)

In [ ]:
cleancityDataDF = cityDataDF.dropna()
cleancityDataDF


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#configure heat map and read key
gmaps.configure(api_key=g_key)

In [ ]:
#heat map of humidity 
locations=cleancityDataDF[["Lat","Lng"]]
humidity=cleancityDataDF["Humidity"]
fig=gmaps.figure()
#fig


#add heat map layer to figure 
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatLayer)
fig

### Create new DataFrame fitting weather criteria


In [ ]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.

#Zero cloudiness.
#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

weatherConditionDF = cleancityDataDF[(cleancityDataDF["Max Temp"] > 70 )& 
                                         (cleancityDataDF["Max Temp"]  < 80) &  
                                         (cleancityDataDF["Wind Speed"] < 10) & 
                                         (cleancityDataDF["Cloudiness"] == 0)]
weatherConditionDF.head()



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = weatherConditionDF.copy()
#hotel_df= hotel_df[["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# create a loop through dataframe
params={
    "radius": 5000,
    "type": "lodging",
    "key":g_key
    }

for index, row in hotel_df.iterrows():
    # get longitude and latitude for each row
    params['location'] = f"{row['Lat']}, {row['Lng']}"
    
    baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(baseURL,params=params).json()
    
    try:
        # populate the array with the hotel name
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
    except: 
        print(f" Error finding lodging in {row['City']}. Skipping...")
        pass
hotel_df


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig